In [82]:
import pandas as pd
import numpy as np
import bs4
import requests
import time
import csv
import xlsxwriter
import xlrd
import openpyxl
import pyjarowinkler as ji
from fuzzywuzzy import fuzz,process

timestr = time.strftime("%Y%m%d-%H%M") # today's date

## initializing the UserAgent object
headers = {"User-Agent" : 'Mozilla/5.0 (Windows NT 10.0; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/76.0.3809.100 Safari/537.36'}

def make_soup(url):
    ## getting the reponse from the page using get method of requests module
    page = requests.get(url, headers=headers)
    
    ## storing the content of the page in a variable
    html = page.content
    
    ## creating BeautifulSoup object
    soup = bs4.BeautifulSoup(html, "html.parser")
    return soup

web_content_list = []
pages = []
web_content_list_plp = []
pages_plp = []

for p in range(1,5):
    urls = 'https://www.primemobileparts.com/iphone-8-plus/c?page='+str(p) 
    souped_links = make_soup(urls)
    for prices1, names1 in zip (souped_links.find_all('div', class_='ecommerce_product_price'), souped_links.find_all('a', class_='ecommerce_product_title')):
        web_content_dict = {}
        web_content_dict["pMp Product"]=names1.text
        web_content_dict["pMp Price"]=prices1.text
        web_content_list.append(web_content_dict)
web_content_list 

df = pd.DataFrame(web_content_list)

df['pMp Price']=df['pMp Price'].str.replace("USD", "")

# below code scrapes product titles and prices from phonelcdparts.com 

for i in range (1,3):
    urls_plp = 'https://www.phonelcdparts.com/iphone-8-plus/?sort=featured&page=' + str(i)
    pages_plp.append(urls_plp)
    
    souped_links_plp = make_soup(urls_plp)

    for prices2, names2 in zip (souped_links_plp.find_all('a', class_='prod-price'), souped_links_plp.find_all('a', class_='pname')):
        web_content_dict_plp = {}
        web_content_dict_plp["pLp Product"]=names2.text
        web_content_dict_plp["pLp Price"]=prices2.text
        web_content_list_plp.append(web_content_dict_plp)
web_content_list_plp 

df_plp = pd.DataFrame(web_content_list_plp)

df_plp['pLp Price']=df_plp['pLp Price'].str.replace("$", "")

df['pLp Product'] = df_plp['pLp Product']
df['pLp Price'] = df_plp['pLp Price']


plp = list(df['pLp Product'].dropna())
pmp = list(df['pMp Product'])


# match the products
# below code is a function to match product.
def match_term(term, list_names, min_score=0):
    max_score = -1
    max_name = " "
    for term2 in list_names:
        score = fuzz.ratio(term, term2)
        if(score>min_score)&(score>max_score):
            max_name = term2
            max_score = score
    return (max_name, max_score)

# use the function to match data
dict_list = []
for name in plp:
    match = match_term(name, pmp, 30)
    
    dict = {}
    dict.update({"plp name" : name})
    dict.update({"pmp_match_name" : match[0]})
    dict.update({"score" : match[1]})
    dict_list.append(dict)
dict_list

# save matched data as dataframe and sort values descending
df_matched = pd.DataFrame(dict_list).sort_values(by='score', ascending=False)

# get only the ones greater than 85. user can change the threshold.
df_matched_ordered = df_matched[df_matched['score']>85]


# get plp prices from the sheet named price list (vlookup)
df_merged = df_matched_ordered.merge(df, left_on = 'plp name', right_on="pLp Product").drop(columns='pMp Product').drop(columns='pMp Price').drop(columns='pLp Product')

#get pmp prices from the sheet named price list (vlookup)
df_merged2 = df_merged.merge(df, left_on = 'pmp_match_name', right_on='pMp Product').drop(columns='pLp Product').drop(columns='pMp Product').drop(columns='pLp Price_y').sort_values(by='score', ascending=False)

df5 = set(df['pMp Product']).intersection(set(df['pLp Product'])) # iki sutunde da olan urunleri bul
df7 = pd.DataFrame(df5) # dataframe olarak kaydet
df8 = df['pLp Product'].append(df7)[0].unique() #kesisim kumesini plp product'in altina ekleyip unique olanlari bul. 
df9 = pd.DataFrame(df8)
df9['Missing'] = pd.DataFrame(df8) # df9 bizde olamayan urunleri gosteriyor 
df10 = df.set_index('pLp Product').join(df9.set_index('Missing')).drop(columns='pMp Product').drop(columns='pMp Price').dropna()

uniqueValues = (df['pLp Product'].append(df['pMp Product'])).unique()
df2 = pd.DataFrame(uniqueValues)

# To write the dataframe to a csv file
writer = pd.ExcelWriter('pMp vs pLp Price Comparision_'+str(timestr)+'.xlsx', engine='openpyxl')
df.to_excel(writer, sheet_name='price list', index=False)
df10.to_excel(writer, sheet_name='missing products', index=False)
df_merged2.to_excel(writer, sheet_name='matched products', index=False)


# df_plp.to_excel(writer, sheet_name='Sheet1', index=False)    
writer.save()
print("Done! Check the created excel file")

Done! Check the created excel file
